In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

### Scrape billboard hot 100

In [ ]:
billboard_top_100 = pd.DataFrame([], columns=['chart_rank', 'artist', 'song_title'])

In [ ]:
def scrape_billboard_hot_100(dataset):
    r = requests.get('https://www.billboard.com/charts/hot-100/')
    soup = BeautifulSoup(r.content, 'html.parser')
    song_rows = soup.find_all('div', attrs={'class': 'o-chart-results-list-row-container'})
    for i, row in enumerate(song_rows):
        song = row.find('h3', attrs={'class': 'a-no-trucate', 'id': 'title-of-a-story'})    
        artist =  row.find_all('span', attrs={'class': 'c-label'})[1]

        dataset.at[dataset.shape[0]] = [i+1, artist.get_text(strip=True), 
                                        song.get_text(strip=True)]

In [ ]:
scrape_billboard_hot_100(billboard_top_100)

In [ ]:
billboard_top_100

In [ ]:
billboard_top_100.to_csv('billboard_100.csv', index=False)

### Expand collection of songs and artists

- Scrape page www.discogs.com
- get items for five music genres: Electronic, Pop, Jazz, Rock, HipHop
- scrape only the first 10 pages (max) with up to 250 items per page 

In [ ]:
HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
GENRES = ['Electronic', 'Pop', 'Jazz', 'Rock', 'Hip+Hop']

In [ ]:
music_database = pd.DataFrame([], columns=['gerne', 'artist', 'song_title'])

In [ ]:
def extract_discogs_list_info(row):
    artist = row.find_all('a')[0].get_text(strip=True)
    song = row.find_all('a')[1].get_text(strip=True)
    return artist, song


def extract_discogs_page_info(url, dataset, genre):
    res = requests.get(url_music, headers=HEADERS)
    soup = BeautifulSoup(res.content, 'html.parser')
    discogs_row = soup.find_all('li', attrs={'role': 'listitem'})
    
    for row in discogs_row:
        dataset.at[dataset.shape[0]] = [genre, *extract_discogs_list_info(row)]

def scrape_discogs_website(list_genres):
    for genre in list_genres:
        for i in range(1, 11):
            url_music = f'https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&ev=gs_mc&layout=sm&genre_exact={genre}&page={i}'
            extract_discogs_page_info(url_music, music_database, genre)

In [ ]:
scrape_discogs_website(GENRES)

In [ ]:
music_database

In [ ]:
music_database.to_csv('music_discogs.csv', index=False)